In [1]:
'''
from google.colab import drive 
drive.mount('/content/drive', force_remount=True)
#'''

"\nfrom google.colab import drive \ndrive.mount('/content/drive', force_remount=True)\n#"

In [2]:
#drive.flush_and_unmount(timeout_ms=24)

In [3]:

import numpy as np
import pandas as pd
import pickle
import numpy as np
import random
import time
import os
#os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = pow(2,40).__str__()
import cv2
from tqdm import tqdm

import tensorflow as tf
from tensorflow.python.keras import Sequential
from tensorflow.keras import layers, optimizers
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from IPython.display import display
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
from keras import optimizers
from sklearn.metrics import classification_report, confusion_matrix
from keras.callbacks import CSVLogger, LambdaCallback
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [29]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
base_dir = 'D:/DATASET_COLLECTION/Dataset_Leaf/MalayaKew_MK/MK/D2/'

train_dir = os.path.join(base_dir, 'train_patch')
test_dir = os.path.join(base_dir, 'test_patch')

#dataset_dir = "drive/My Drive/Plant_Leaf_MalayaKew_MK_Dataset/MK/D2/"
work_dir = "D:/DATASET_COLLECTION/Dataset_Leaf/MalayaKew_MK/Records/"
#data_instance = 64 # 64 256
color_type = 'rgb' # rgb, grayscale

In [5]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
BATCH_SIZE = 4

In [7]:
train_generator = train_datagen.flow_from_directory(
        train_dir,
        batch_size=BATCH_SIZE,
        color_mode=color_type, # grayscale, rgb
        class_mode='categorical')

Found 34672 images belonging to 44 classes.


In [8]:
num_classes = train_generator.num_classes
total_train_data = train_generator.samples

print(f"total_train_data = {total_train_data}")
print(f"train_generator.image_shape = {train_generator.image_shape}")
print(f"num_classes = {num_classes}")

total_train_data = 34672
train_generator.image_shape = (256, 256, 3)
num_classes = 44


In [9]:
test_generator = test_datagen.flow_from_directory(
        test_dir,
        batch_size=BATCH_SIZE,
        color_mode=color_type, # grayscale, rgb
        class_mode='categorical')

Found 8800 images belonging to 44 classes.


In [10]:
total_test_data = test_generator.samples
print(f"total_test_data = {total_test_data}")

total_test_data = 8800


In [11]:
# DenseNet121 ResNet101 ResNet50 DenseNet201 InceptionV3 Xception NASNetLarge ResNet152V2 InceptionResNetV2 EfficientNetB7
impl_type = "Custom." # TransferLearning3D, Custom
dataset = f"MalayaKew.Original.{color_type}.{train_generator.image_shape[:2]}.DataFlow" # +str(img_size)+"p"

In [12]:
#'''
count_no_improvement = 0
epoch_initial = True
#'''

In [13]:
NUM_NEURONS = 32
#NUM_LAYERS = 3
#BATCH_SIZE = 16 # 10
NUM_EPOCHS = 300
epochs_completed = 0
LEARNING_RATE = 0.00001
EPSILON = 1e-4
early_stop_after_epochs = 20
DROPOUT = 0.5 # 0.5 0.0
pad = 0

LOSS = 'categorical_crossentropy'
ACTIVATION_FUNCTION = 'elu' # relu sigmoid elu
FINAL_ACTIVATION_FUNCTION = 'softmax'
validation_split = 0.1
kernel_size=(1,1)
pointTrainableAfter = "allDefault" # "allDefault" 160 170
OPTIMIZER = "Adam" # Adam SGD RMSProp
init_weights = "" # "imagenet" None
modelExt = ".Dense.512.1024.2048" # .Dense.128.256.512, .512.512.512, .Dense.512.512.512.512.Res, .Dense.512.1024.2048
l2_val = 0.001 # 0.001, 

# +"_kernel"+str(kernel_size)+"_lr"+str(LEARNING_RATE)+"_batch"+str(BATCH_SIZE)+"_epochs"+str(NUM_EPOCHS)
#checkpointer_name  = "weights_"+dataset+"_"+impl_type+"_nLayers"+str(NUM_LAYERS)+"_nNeurons"+str(NUM_NEURONS)+".hdf5"
ext = f".Flatten.run_1" # run_1, run_2, .DropAfter, .momentum0.9, .Flatten.l2.{str(l2_val)}
#'''
checkpointer_name  = "weights."+dataset+".pad"+str(pad)+"."+impl_type+".wInit."+str(init_weights)+".TrainableAfter."+str(pointTrainableAfter)+\
                     modelExt+".actF."+ACTIVATION_FUNCTION+".opt."+OPTIMIZER+".drop."+str(DROPOUT)+".batch"+str(BATCH_SIZE)+ext+".hdf5"
log_name = "log."+checkpointer_name[8:-5]+".log"

print('checkpointer_name =', checkpointer_name)
print('log_name =', log_name)
#'''

checkpointer_name = weights.MalayaKew.Original.rgb.(256, 256).DataFlow.pad0.Custom..wInit..TrainableAfter.allDefault.Dense.512.1024.2048.actF.elu.opt.Adam.drop.0.5.batch4.Flatten.run_1.hdf5
log_name = log.MalayaKew.Original.rgb.(256, 256).DataFlow.pad0.Custom..wInit..TrainableAfter.allDefault.Dense.512.1024.2048.actF.elu.opt.Adam.drop.0.5.batch4.Flatten.run_1.log


In [14]:
print(train_generator.image_shape)

(256, 256, 3)


In [15]:
"""
#base_model=DenseNet121(weights=None, include_top=False, input_shape=np_train_dataset2.shape[1:]) # `None` (random initialization)
#base_model=ResNet152V2(weights=None, include_top=False, input_shape=np_train_dataset2.shape[1:])
# ResNet152V2 ResNet50 ResNet101 ResNet152 DenseNet201 InceptionV3 Xception NASNetLarge 'imagenet' ResNet152V2 DenseNet121
#inputs = Input(final_train_imageset.shape[1:])
#x = ZeroPadding2D(padding=(pad,pad))(inputs)
#base_model=tf.keras.applications.ResNet50(weights=init_weights, include_top=False, input_tensor=x)
base_model=tf.keras.applications.DenseNet201(weights=init_weights, include_top=False, input_shape=train_generator.image_shape)
x=base_model.output

x = Flatten()(x)

'''
x = Dense(512, kernel_regularizer=tf.keras.regularizers.l2(l2_val), activation=ACTIVATION_FUNCTION)(x)
#x_copy = x
x = Dropout(DROPOUT)(x)
x = Dense(1024, kernel_regularizer=tf.keras.regularizers.l2(l2_val), activation=ACTIVATION_FUNCTION)(x)
x = Dropout(DROPOUT)(x)
x = Dense(2048, kernel_regularizer=tf.keras.regularizers.l2(l2_val), activation=ACTIVATION_FUNCTION)(x)
x = Dropout(DROPOUT)(x)
#x = Add()([x,x_copy])
#'''
outputs=Dense(num_classes,activation='softmax')(x)

model=Model(inputs=base_model.input,outputs=outputs)
model.summary()
#"""

"\n#base_model=DenseNet121(weights=None, include_top=False, input_shape=np_train_dataset2.shape[1:]) # `None` (random initialization)\n#base_model=ResNet152V2(weights=None, include_top=False, input_shape=np_train_dataset2.shape[1:])\n# ResNet152V2 ResNet50 ResNet101 ResNet152 DenseNet201 InceptionV3 Xception NASNetLarge 'imagenet' ResNet152V2 DenseNet121\n#inputs = Input(final_train_imageset.shape[1:])\n#x = ZeroPadding2D(padding=(pad,pad))(inputs)\n#base_model=tf.keras.applications.ResNet50(weights=init_weights, include_top=False, input_tensor=x)\nbase_model=tf.keras.applications.DenseNet201(weights=init_weights, include_top=False, input_shape=train_generator.image_shape)\nx=base_model.output\n\nx = Flatten()(x)\n\n'''\nx = Dense(512, kernel_regularizer=tf.keras.regularizers.l2(l2_val), activation=ACTIVATION_FUNCTION)(x)\n#x_copy = x\nx = Dropout(DROPOUT)(x)\nx = Dense(1024, kernel_regularizer=tf.keras.regularizers.l2(l2_val), activation=ACTIVATION_FUNCTION)(x)\nx = Dropout(DROPOUT)(x

In [16]:
img_shape = train_generator.image_shape
print(f"img_shape = {img_shape}")

img_shape = (256, 256, 3)


In [17]:
#""" 25,690,112; 1,492,992; 1,843,200; 819,200; 51,200
inputs = Input(train_generator.image_shape)
x = inputs
#x = ZeroPadding2D(padding=(pad,pad))(inputs)

for _ in range(60):
    x = Conv2D(NUM_NEURONS, kernel_size=(3,3), activation=ACTIVATION_FUNCTION)(x)
    x = MaxPooling2D(pool_size=(3,3), strides=(1,1))(x)

x = Flatten()(x)
x = Dense(1024, kernel_regularizer=tf.keras.regularizers.l2(l2_val), activation=ACTIVATION_FUNCTION)(x)
x = Dropout(DROPOUT)(x)
x = Dense(1024, kernel_regularizer=tf.keras.regularizers.l2(l2_val), activation=ACTIVATION_FUNCTION)(x)
x = Dropout(DROPOUT)(x)
x = Dense(2048, kernel_regularizer=tf.keras.regularizers.l2(l2_val), activation=ACTIVATION_FUNCTION)(x)
x = Dropout(DROPOUT)(x)

outputs=Dense(num_classes,activation='softmax')(x)

model=Model(inputs=inputs,outputs=outputs)
model.summary()
#"""

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 252, 252, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 250, 250, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 248, 248, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 246, 246, 32)      9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 244, 244, 32)      0     

In [18]:
'''
tf.keras.utils.plot_model(
    model, to_file='model.png', show_shapes=True, show_dtype=False,
    show_layer_names=True, rankdir='TB', expand_nested=True, dpi=64
)
#'''

"\ntf.keras.utils.plot_model(\n    model, to_file='model.png', show_shapes=True, show_dtype=False,\n    show_layer_names=True, rankdir='TB', expand_nested=True, dpi=64\n)\n#"

In [19]:
count_trainable = 0
count_non_trainable = 0

#'''
if pointTrainableAfter == "allDefault":
    for layer in model.layers:
        layer.trainable=True
        count_trainable += 1
elif pointTrainableAfter > 0:
    for layer in model.layers[:pointTrainableAfter]: # [:-pointTrainableAfter]
        layer.trainable=False
        count_non_trainable += 1
    for layer in model.layers[pointTrainableAfter:]: # [-pointTrainableAfter:]
        layer.trainable=True
        count_trainable += 1
#'''

'''
for layer in model.layers:
    layer.trainable=True
    count_trainable += 1
#'''

print("count_non_trainable =", count_non_trainable)
print("count_trainable =", count_trainable)
print("Total number of layers =", count_non_trainable+count_trainable)

count_non_trainable = 0
count_trainable = 129
Total number of layers = 129


In [20]:
'''
checkpointer_name  = "weights."+dataset+".pad"+str(pad)+"."+impl_type+".wInit."+str(init_weights)+".TrainableAfter."+str(pointTrainableAfter)+\
                     modelExt+".opt."+OPTIMIZER+".drop."+str(DROPOUT)+".batch"+str(BATCH_SIZE)+ext+".hdf5"
log_name = "log."+checkpointer_name[8:-5]+".log"

print('checkpointer_name =', checkpointer_name)
print('log_name =', log_name)
#'''

'\ncheckpointer_name  = "weights."+dataset+".pad"+str(pad)+"."+impl_type+".wInit."+str(init_weights)+".TrainableAfter."+str(pointTrainableAfter)+                     modelExt+".opt."+OPTIMIZER+".drop."+str(DROPOUT)+".batch"+str(BATCH_SIZE)+ext+".hdf5"\nlog_name = "log."+checkpointer_name[8:-5]+".log"\n\nprint(\'checkpointer_name =\', checkpointer_name)\nprint(\'log_name =\', log_name)\n#'

In [21]:
#run_opts = tf.compat.v1.RunOptions(report_tensor_allocations_upon_oom = True)

In [22]:
# "RMSProp" "SGD" "Adam" "Adamax" "Adadelta" "Adagrad" "SGD"
#optimizer = tf.keras.optimizers.RMSprop(lr = LEARNING_RATE, epsilon=EPSILON)

if OPTIMIZER == "RMSProp":
    optimizer = tf.keras.optimizers.RMSprop(lr = LEARNING_RATE, epsilon=EPSILON)
elif OPTIMIZER == "Adam":
    optimizer = tf.keras.optimizers.Adam(lr = LEARNING_RATE, epsilon=EPSILON, beta_1=0.9, beta_2=0.999)
elif OPTIMIZER == "Adamax":
    optimizer = tf.keras.optimizers.Adamax(lr = LEARNING_RATE, epsilon=EPSILON, beta_1=0.9, beta_2=0.999)
elif OPTIMIZER == "Adadelta":
    optimizer = tf.keras.optimizers.Adadelta(lr = LEARNING_RATE, epsilon=EPSILON, rho=0.95)
elif OPTIMIZER == "Adagrad":
    optimizer = tf.keras.optimizers.Adagrad(lr = LEARNING_RATE, epsilon=EPSILON, initial_accumulator_value=0.1)
elif OPTIMIZER == "SGD":
    optimizer = tf.keras.optimizers.SGD(lr = LEARNING_RATE, momentum=0.9)

model.compile(
    #optimizer=OPTIMIZER,
    optimizer=optimizer,
    loss=LOSS,
    metrics=['accuracy']
)

print("OPTIMIZER =", OPTIMIZER)

OPTIMIZER = Adam


In [23]:
# save the best model with least validation loss
checkpointer = ModelCheckpoint(filepath = work_dir+checkpointer_name, 
                               #monitor='val_accuracy',
                               monitor='val_loss',
                               save_weights_only=False,  
                               mode='auto', 
                               verbose = 0, # 0 = silent, 1 = progress bar, 2 = one line per epoch
                               save_best_only =False
                               )
checkpointer_best = ModelCheckpoint(filepath = work_dir+"best_"+checkpointer_name, 
                                    monitor='val_loss', 
                                    save_weights_only=False,
                                    mode='auto',  
                                    verbose = 1, 
                                    save_best_only = True
                                    )
early_stopping = EarlyStopping(monitor='loss', patience=early_stop_after_epochs)

In [24]:
'''
if 'count_no_improvement' not in globals():
    count_no_improvement = 0
    print("count_no_improvement =", count_no_improvement)
#'''
'''
count_no_improvement = 0
epoch_initial = False
#'''
min_delta = 0.0009
print("count_no_improvement =", count_no_improvement)

def checkBestPerformance(epoch, logs):
    save_filepath = work_dir+"best_"+checkpointer_name

    global epoch_initial
    if epoch_initial == True:
        epoch_initial = False
        model.save(filepath = save_filepath)
        print(". Model saved!")

    elif epoch_initial == False:
        global count_no_improvement

        log_data = pd.read_csv(work_dir+log_name, sep=',', usecols=['val_loss', 'val_accuracy'], engine='python')
        min_val_loss = float(str(min(log_data.val_loss.values))[:6])
        max_val_acc = float(str(max(log_data.val_accuracy.values))[:6])

        current_val_acc = float(str(logs['val_accuracy'])[:6])
        current_val_loss = float(str(logs['val_loss'])[:6])

        if (current_val_loss < min_val_loss) and (abs(current_val_loss-min_val_loss) >= min_delta):
            count_no_improvement = 0
            model.save(filepath = save_filepath)
            print("\nval_loss decreased from",min_val_loss," to",current_val_loss,"( val_accuracy =",current_val_acc,").")

        elif (current_val_loss==min_val_loss) and (current_val_acc>max_val_acc):
            count_no_improvement = 0
            model.save(filepath = save_filepath)
            print("\nval_accuracy increased to", current_val_acc, ".")

        else:
            count_no_improvement += 1
            print(". count_no_improvement =", count_no_improvement)

        if count_no_improvement >= early_stop_after_epochs:
            global list_callbacks
            del list_callbacks, count_no_improvement
            #print("count_no_improvement =", count_no_improvement, "... list_callbacks =", list_callbacks)

count_no_improvement = 0


In [25]:
epochs_completed = 0
list_callbacks = []
csv_logger = CSVLogger(work_dir+log_name, separator=',', append=True)

#if 'list_callbacks' in globals():
#    del list_callbacks

try:
    log_data = pd.read_csv(work_dir+log_name, sep=',', usecols=['epoch'], engine='python')
    epochs_completed = log_data.shape[0]

    #if epochs_completed > 0:
    model = load_model(work_dir+checkpointer_name)
    list_callbacks = [checkpointer, LambdaCallback(on_epoch_end=checkBestPerformance), csv_logger]
    print("epochs_completed =", epochs_completed)

except Exception as error:
    if epochs_completed == 0:
        # list_callbacks = [checkpointer, checkpointer_best, csv_logger, early_stopping] 
        list_callbacks = [checkpointer, LambdaCallback(on_epoch_end=checkBestPerformance), csv_logger]
        print("epochs_completed =", epochs_completed)
    elif epochs_completed > 0:
        print(error)

print('checkpointer_name =', checkpointer_name)

epochs_completed = 0
checkpointer_name = weights.MalayaKew.Original.rgb.(256, 256).DataFlow.pad0.Custom..wInit..TrainableAfter.allDefault.Dense.512.1024.2048.actF.elu.opt.Adam.drop.0.5.batch4.Flatten.run_1.hdf5


In [26]:
import PIL
from PIL import Image

In [27]:
print('checkpointer_name =', checkpointer_name)
print("Previously completed epochs =", epochs_completed)
print("count_no_improvement =", count_no_improvement, "\n")

#'''
try:
    start_time = time.time()
    history = model.fit(train_generator, 
                        #steps_per_epoch=total_train_data // BATCH_SIZE,
                        batch_size=BATCH_SIZE,
                        shuffle=True, 
                        epochs = NUM_EPOCHS - epochs_completed, 
                        validation_data=test_generator,
                        #validation_steps=total_test_data // BATCH_SIZE,
                        callbacks=list_callbacks
                        )
    elapsed_time = time.time() - start_time 
    print("\nTime elapsed: ", elapsed_time)

except Exception as error:
    print("\nError:", error)
#'''

checkpointer_name = weights.MalayaKew.Original.rgb.(256, 256).DataFlow.pad0.Custom..wInit..TrainableAfter.allDefault.Dense.512.1024.2048.actF.elu.opt.Adam.drop.0.5.batch4.Flatten.run_1.hdf5
Previously completed epochs = 0
count_no_improvement = 0 

Epoch 1/300

Error:  OOM when allocating tensor with shape[4,32,134,134] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model/conv2d_30/Conv2D (defined at <ipython-input-27-864238292d91>:8) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_6222]

Function call stack:
train_function



In [28]:
'''
Record: 
;
---
Test Acc: 0.3947, Test Loss: 3.1064: ep10, weights.MalayaKew.gray.pad10.TransferLearning.InceptionResNetV2.wInit.None.TrainableAfter.allDefault.Dense.512.actF.relu.opt.Adam.drop.0.0.batch64.Flatten.run_1.hdf5
Test Acc: 0.6353, Test Loss: 1.8520: ep30, weights.MalayaKew.gray3D.(64, 64, 3).pad0.TransferLearning3D.ResNet50.wInit.imagenet.TrainableAfter.allDefault.actF.relu.opt.Adam.drop.0.0.batch64.Flatten.run_1.hdf5
Test Acc: 0.4453, Test Loss: 2.4178: ep17, weights.MalayaKew.gray3D.(64, 64, 3).pad0.TransferLearning3D.ResNet50.wInit.imagenet.TrainableAfter.170.actF.relu.opt.Adam.drop.0.0.batch64.Flatten.run_1.hdf5
Test Acc: 0.6044, Test Loss: 2.3383: ep22, weights.MalayaKew.gray3D.(64, 64, 3).pad0.TransferLearning3D.ResNet50.wInit.imagenet.TrainableAfter.allDefault.Dense.512.512.512.512.actF.elu.opt.Adam.drop.0.5.batch64.Flatten.regularizers.l2.0.0001.run_1.hdf5
Test Acc: 0.6112, Test Loss: 2.5392: ep24, weights.MalayaKew.gray3D.(64, 64, 3).pad0.TransferLearning3D.ResNet50.wInit.imagenet.TrainableAfter.allDefault.Dense.512.512.512.512.Res.actF.elu.opt.Adam.drop.0.5.batch64.Flatten.regularizers.l2.0.0001.run_1.hdf5
Test Acc: 0.3967, Test Loss: 4.1926: ep25, weights.MalayaKew.gray3D.(64, 64, 3).pad0.TransferLearning3D.ResNet152V2.wInit.imagenet.TrainableAfter.allDefault.actF.relu.opt.Adam.drop.0.0.batch64.Flatten.run_1.hdf5

#'''

csv_logger = CSVLogger(work_dir+log_name, separator=',', append=True)
log_data = pd.read_csv(work_dir+log_name, sep=',', usecols=['epoch'], engine='python')
epochs_completed = log_data.shape[0]

result = model.evaluate(final_test_imageset, final_test_label)
print("Test Acc: {}, Test Loss: {}: ep{}, {}\n".format(round(result[1],4), round(result[0],4), epochs_completed, checkpointer_name))


EmptyDataError: No columns to parse from file

In [ ]:
#checkpointer_name = "weights.Fashion.DenseNet121.wInit.None.TrainableAfterallDefault.opt.SGD.drop.0.0.batch32.Flatten.run_1.hdf5"
model_loaded = load_model(work_dir+"best_"+checkpointer_name)
print("Loaded "+work_dir+"best_"+checkpointer_name+".")

In [ ]:
'''
Record: 
;
---
Test Acc: 0.3833, Test Loss: 2.1568: ep10, best_weights.MalayaKew.gray.pad10.TransferLearning.InceptionResNetV2.wInit.None.TrainableAfter.allDefault.Dense.512.actF.relu.opt.Adam.drop.0.0.batch64.Flatten.run_1.hdf5
Test Acc: 0.7168, Test Loss: 1.2828: ep30, best_weights.MalayaKew.gray3D.(64, 64, 3).pad0.TransferLearning3D.ResNet50.wInit.imagenet.TrainableAfter.allDefault.actF.relu.opt.Adam.drop.0.0.batch64.Flatten.run_1.hdf5

Test Acc: 0.4410, Test Loss: 2.0524: ep17, best_weights.MalayaKew.gray3D.(64, 64, 3).pad0.TransferLearning3D.ResNet50.wInit.imagenet.TrainableAfter.170.actF.relu.opt.Adam.drop.0.0.batch64.Flatten.run_1.hdf5
Test Acc: 0.6757, Test Loss: 1.5270: ep22, best_weights.MalayaKew.gray3D.(64, 64, 3).pad0.TransferLearning3D.ResNet50.wInit.imagenet.TrainableAfter.allDefault.Dense.512.512.512.512.actF.elu.opt.Adam.drop.0.5.batch64.Flatten.regularizers.l2.0.0001.run_1.hdf5
Test Acc: 0.5914, Test Loss: 1.7184: ep24, best_weights.MalayaKew.gray3D.(64, 64, 3).pad0.TransferLearning3D.ResNet50.wInit.imagenet.TrainableAfter.allDefault.Dense.512.512.512.512.Res.actF.elu.opt.Adam.drop.0.5.batch64.Flatten.regularizers.l2.0.0001.run_1.hdf5
Test Acc: 0.6594, Test Loss: 1.2659: ep25, best_weights.MalayaKew.gray3D.(64, 64, 3).pad0.TransferLearning3D.ResNet152V2.wInit.imagenet.TrainableAfter.allDefault.actF.relu.opt.Adam.drop.0.0.batch64.Flatten.run_1.hdf5

#'''
'''
csv_logger = CSVLogger(work_dir+log_name, separator=',', append=True)
log_data = pd.read_csv(work_dir+log_name, sep=',', usecols=['epoch'], engine='python')
epochs_completed = log_data.shape[0]
#'''

result2 = model_loaded.evaluate(final_test_imageset, final_test_label)
#print("nLayers: {}, nNeurons: {}, DROPOUT: {}, Test Acc: {}, Test Loss: {}".format(NUM_LAYERS, NUM_NEURONS, DROPOUT, round(result2[1], 4), round(result2[0], 4)))
print("Test Acc: {}, Test Loss: {}: ep{}, {}\n".format(round(result2[1],4), round(result2[0],4), epochs_completed, "best_"+checkpointer_name))


In [ ]:
import csv

with open(work_dir+'Records.csv', "a") as fp:
    wr = csv.writer(fp, dialect='excel')
    try:
        wr.writerow([checkpointer_name[8:-5], round(result2[1], 4), round(result2[0], 4), elapsed_time])
    except:
        wr.writerow([checkpointer_name[8:-5], round(result2[1], 4), round(result2[0], 4)])
print("Saved results.")

In [ ]:
#Confution Matrix and Classification Report
#'''
Y_pred = model_loaded.predict_generator(final_test_imageset, len(final_test_imageset))
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(np_test_label, y_pred))
#'''

In [ ]:
# Precision [TP/TP+FP] = The ratio of correctly predicted positive observations to the total predicted positive observations.
# Recall (Sensitivity) [TP/TP+FN] = The ratio of correctly predicted positive observations to the all observations in actual class - 'yes'.
# F1 score [F1 Score = 2*(Recall * Precision) / (Recall + Precision)] = The weighted average of Precision and Recall.
# Support = The number of samples of the true response that lie in that class.
#'''
print('Classification Report:')
#target_names = ['Mono', 'Di'] # not ['Di', 'Mono']
print(classification_report(np_test_label, y_pred)) #, target_names=target_names))
#'''

In [ ]:
'''
print('Classification Report')
print(classification_report(y_test, y_pred, target_names=list_categories))
#'''

In [ ]:
log_data = pd.read_csv(work_dir+log_name, sep=',', engine='python') 

In [ ]:
# Getting the model history keys 
#history.history.keys()
log_data.head()

In [ ]:
# plot the training artifacts
title = "Val loss for "+dataset+" "+impl_type

plt.plot(log_data['loss'])
plt.plot(log_data['val_loss'])
plt.title(title)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train_loss','val_loss'], loc = 'best')

img_path = work_dir+'Images/vLoss_'+checkpointer_name[8:-5]+'.png'
plt.savefig(img_path, dpi=600)
plt.show()
print('img_path =', img_path)

In [ ]:
title = "Val acc for "+dataset+" "+impl_type

plt.plot(log_data['accuracy'])
plt.plot(log_data['val_accuracy'])
plt.title(title)
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train_accuracy','val_accuracy'], loc = 'best')

img_path = work_dir+'Images/vAcc_'+checkpointer_name[8:-5]+'.png'
plt.savefig(img_path, dpi=600)
plt.show()
print('img_path =', img_path)